# Import all the libraries

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
#from sklearn_genetic import GASearchCV
from sklearn_genetic import GAFeatureSelectionCV
#from sklearn_genetic.space import Categorical, Integer, Continuous
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
#from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.cross_decomposition import PLSCanonical
from xgboost import XGBClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import *
import matplotlib.pyplot as plt
import pickle
import numpy as np
import time
#from tpot import TPOTClassifier
import shap

from sklearn.metrics import accuracy_score
#import tensorflow as tf
#from autogluon.tabular import TabularDataset, TabularPredictor
#import keras 
#from keras.losses import BinaryCrossentropy
from tpot import TPOTClassifier
from sklearn.ensemble import VotingClassifier

import subprocess # TO RUN VBEOSA

#from scikeras.wrappers import KerasClassifier # TO USE VOTING

from EKAN_functions import *
import statsmodels.api as sm
from sklearn.model_selection import train_test_split


from tabtransformertf.models.fttransformer import FTTransformerEncoder, FTTransformer

from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split
import torch


In [ ]:

## DEBUGGER ## TEST WETHER THE CV INDEXES ARE THE SAME
#cv1 = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=1)
#cv2 = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=1)
#train2_tot=[]
#for train2,test2 in cv2.split(feats,target):
#    train2_tot.append(train2)

#i=0
#for train1,test1 in cv1.split(feats,target):
#    for index in range(0,len(train1)):
#        if train1[index] != train2_tot[i][index]:
#            print('PROBLEM')
#    i+=1
        


# Import Data

In [ ]:
# Get all the real outer loop for shap

X_gs = pd.read_csv('your_corrected_folds_x.csv')
X_gs = X_gs.drop(columns=['Unnamed: 0'],axis=1)
#features=X_gs.columns[0:9]


#X_gs = X_gs.drop(columns=features,axis=1)


y_gs = pd.read_csv('your_corrected_folds_y.csv')
y_gs = y_gs.drop(columns=['Unnamed: 0'],axis=1)



with open("your_folds_pointers.pickle", "rb") as f:
    cv_gs_tot = pickle.load(f)
f.close()

### NESTED CV LOOP (corrected in the loop)

In [ ]:
def my_CV_KN(m,k1,k2,N):
    accuracy_means = []
    accuracy_vars = []
    auc_means = []
    auc_vars = []
    f1_means = []
    f1_vars= []
    Runtime_count={}
    
    # Get all the real outer loop for shap

    # Get all the real outer loop for shap

    X_gs = pd.read_csv('your_corrected_folds_x.csv')
    X_gs = X_gs.drop(columns=['Unnamed: 0'],axis=1)
    #features=X_gs.columns[0:9]


    #X_gs = X_gs.drop(columns=features,axis=1)


    y_gs = pd.read_csv('your_corrected_folds_y.csv')
    y_gs = y_gs.drop(columns=['Unnamed: 0'],axis=1)



    with open("your_folds_pointers.pickle", "rb") as f:
        cv_gs_tot = pickle.load(f)
    f.close()


    for k in [400]:
        outer_results_acc = []
        outer_results_auc = []
        outer_results_f1 = [] 
        fold_number=0
        Runtime_count[k]=0 
        while fold_number<10:
            scores_inner=[]
            # ESTRACT CORRECTED OUTER FOLD

            X_tr,X_v=X_gs.iloc[cv_gs_tot[fold_number][0]],X_gs.iloc[cv_gs_tot[fold_number][1]]
            y_tr,y_v=y_gs.iloc[cv_gs_tot[fold_number][0]],y_gs.iloc[cv_gs_tot[fold_number][1]]

            # TRY AND CATCH RUNTIMES ERROR OF KANS
            try:
                models,model_p,dataset,dataset_p=KAN_tr_m(X_tr,y_tr.astype('int64'),X_v,y_v.astype('int64'),k,random_s=True,m=m,fil=True,subs=False,k1=k1,k2=k2,N=N)
                # PREDICT
                if m=='KAN' or m=='KAN_1':
                    y_pred=torch.argmax(model_p(dataset_p['test_input']),dim=1)#model.predict(X_v.values) #ensemble_predictions(model,X_v)

                if m=='MLP' or m=='SVM':
                    y_pred=model_p.predict(dataset_p)


                # ESTIMATING OUTER FOLD METRICS
                acc = accuracy_score(y_v, y_pred)
                #bal_acc=balanced_accuracy_score(y_v, y_pred)
                f1=f1_score(y_v,y_pred)
                auc=roc_auc_score(y_v,y_pred)
                # store the result
                outer_results_acc.append(acc)
                #outer_results_bal_acc.append(bal_acc)
                outer_results_f1.append(f1)
                outer_results_auc.append(auc)
                # report progress
                print('>acc=%.3f' % (acc))
                #print('>bal_accacc=%.3f' % (bal_acc))
                print('>F1=%.3f' % (f1))
                print('>auc=%.3f' % (auc))
                Runtime_count[k]=0       
            except RuntimeError:
                    Runtime_count[k]=Runtime_count[k]+1
                    #fold_number=fold_number-1


            fold_number+=1
        # summarize the estimated performance of the model
        # Assuming outer_results_acc, outer_results_bal_acc, and outer_results_f1 are already defined
        accuracy_means.append(np.mean(outer_results_acc))
        accuracy_vars.append(np.std(outer_results_acc))

        # Append the mean and standard deviation of balanced accuracy results
        auc_means.append(np.mean(outer_results_auc))
        auc_vars.append(np.std(outer_results_auc))

        # Append the mean and standard deviation of F1 score results
        f1_means.append(np.mean(outer_results_f1))
        f1_vars.append(np.std(outer_results_f1))
        print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results_acc), np.std(outer_results_acc)))
        print('Accuracy results refer to mean and variance')

        print('AUC: %.3f (%.3f)' % (np.mean(outer_results_auc), np.std(outer_results_auc)))
        print('AUC results refer to mean and variance')

        print('F1: %.3f (%.3f)' % (np.mean(outer_results_f1), np.std(outer_results_f1)))
        print('F1 results refer to mean and variance')


    return [[accuracy_means,accuracy_vars],[auc_means,auc_vars],[f1_means,f1_vars]]

In [ ]:
import itertools
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score

# K1 vs N

In [ ]:
# Define your arrays
k1=[4,10,20,50,70,100]
k2=[4,7,10,15,17,20]
N=[2,4,6,8,15,20]
# Generate all possible pairs
pairs = list(itertools.product(k1, N))

# Initialize the dictionary to store pair values
pair_values_2 = {}

# Assign a random value to each pair
for pair in pairs:
    [[accuracy_means,accuracy_vars],[auc_means,auc_vars],[f1_means,f1_vars]]=my_CV_KN('KAN',k1=pair[0],k2=4,N=pair[1])
    pair_values_2[pair] = f1_means



In [ ]:
# Initialize the heatmap matrix
heatmap_matrix = np.zeros((len(k1), len(N)))

# Fill the heatmap matrix with the values from pair_values
for i, x in enumerate(k1):
    for j, y in enumerate(N):
        heatmap_matrix[i, j] = pair_values_2.get((x, y), 0)[0]

# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(heatmap_matrix, annot=True, xticklabels=N, yticklabels=k1, cmap="Greens")
plt.xlabel('N')
plt.ylabel('k1')
plt.title('Heatmap of k1 vs N, k2=20')

# Show the plot
plt.show()

# K1 vs K2

In [ ]:
# Define your arrays
k1=[4,10,20,50,70,100]
k2=[4,7,10,15,17,20]
N=[2,4,6,8,15,20]
# Generate all possible pairs
pairs = list(itertools.product(k1, k2))

# Initialize the dictionary to store pair values
pair_values_1 = {}

# Assign a random value to each pair
for pair in pairs:
    [[accuracy_means,accuracy_vars],[auc_means,auc_vars],[f1_means,f1_vars]]=my_CV_KN('KAN',k1=pair[0],k2=pair[1],N=20)
    pair_values_1[pair] = f1_means

In [ ]:
# Initialize the heatmap matrix
heatmap_matrix = np.zeros((len(k1), len(k2)))

# Fill the heatmap matrix with the values from pair_values
for i, x in enumerate(k1):
    for j, y in enumerate(k2):
        heatmap_matrix[i, j] = pair_values_1.get((x, y), 0)[0]

# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(heatmap_matrix, annot=True, xticklabels=k2, yticklabels=k1,cmap="YlGnBu")
plt.xlabel('k2')
plt.ylabel('k1')
plt.title('Heatmap of k1 vs k2, N=20')

# Show the plot
plt.show()

# k2 vs N

In [ ]:
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

# Define your arrays
k1=[4,10,20,50,70,100]
k2=[4,7,10,15,17,20]
N=[2,4,6,8,15,20]

# Generate all possible pairs
pairs = list(itertools.product(k2, N))

# Initialize the dictionary to store pair values
pair_values = {}

# Assign a random value to each pair
for pair in pairs:
    [[accuracy_means,accuracy_vars],[auc_means,auc_vars],[f1_means,f1_vars]]=my_CV_KN('KAN',k1=4,k2=pair[0],N=pair[1])
    pair_values[pair] = f1_means




In [ ]:
# Initialize the heatmap matrix
heatmap_matrix = np.zeros((len(k2), len(N)))

# Fill the heatmap matrix with the values from pair_values
for i, x in enumerate(k2):
    for j, y in enumerate(N):
        heatmap_matrix[i, j] = pair_values.get((x, y), 0)[0]

# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(heatmap_matrix, annot=True, xticklabels=N, yticklabels=k2, cmap='Reds')
plt.xlabel('N')
plt.ylabel('k2')
plt.title('Heatmap of k2 vs N,k1=4')

# Show the plot
plt.show()

# COMPARISONS

In [ ]:
from sklearn.decomposition import KernelPCA,PCA,FastICA

from sklearn.metrics import balanced_accuracy_score
import random

outer_results_acc = []
outer_results_auc = []
outer_results_f1 = [] 


# Get all the real outer loop for shap

X_gs = pd.read_csv('your_corrected_folds_x.csv')
X_gs = X_gs.drop(columns=['Unnamed: 0'],axis=1)
#features=X_gs.columns[0:9]


#X_gs = X_gs.drop(columns=features,axis=1)


y_gs = pd.read_csv('your_corrected_folds_y.csv')
y_gs = y_gs.drop(columns=['Unnamed: 0'],axis=1)



with open("your_folds_pointers.pickle", "rb") as f:
    cv_gs_tot = pickle.load(f)
f.close()


import itertools
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_auc_score
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
fold_number=0
while fold_number<10: # run across all the raw train and test splits

    scores_inner=[]
    # ESTRACT CORRECTED OUTER FOLD

    X_tr,X_v=X_gs.iloc[cv_gs_tot[fold_number][0]],X_gs.iloc[cv_gs_tot[fold_number][1]]
    y_tr,y_v=y_gs.iloc[cv_gs_tot[fold_number][0]],y_gs.iloc[cv_gs_tot[fold_number][1]]

    

        
    # UNCOMMENT MODEL TO GET COMPARISONS

    ## UNCOMMENT TO VALIDATE SKLEARN MODELS AND XGB

    #models,model_p,dataset,dataset_p=KAN_tr_m(X_tr,y_tr.astype('int64'),X_v,y_v.astype('int64')) # int64 for type long


    #model=RandomForestClassifier(n_estimators=20)#
    #model=AdaBoostClassifier()#XGBClassifier()#svm.SVC()
    #model=model.fit(X_train_gs_inner,y_train_gs_inner)
    #model=model.fit(X_tr,y_tr)

    ## UNCOMMENT TO VALIDATE TABNET

    #X_train=X_tr.values
    #y_train=y_tr.values[:,0]
    #X_test=X_v.values
    #y_test=y_v.values[:,0]

    # define the model
    #model= TabNetClassifier(n_steps=10,optimizer_fn=torch.optim.Adam, lambda_sparse=1e-4, momentum=0.3,
                            #scheduler_params={"step_size":10, 
                                            #"gamma":0.9},
                            #scheduler_fn=torch.optim.lr_scheduler.StepLR,
                            #)



    # fit the model 
    #model.fit(
        #X_train,y_train,
        #eval_set=[(X_train, y_train), (X_test, y_test)],
        #eval_name=['train', 'test'],
        #eval_metric=['accuracy','balanced_accuracy'],
        #max_epochs=200, patience=300,
        #batch_size=52, virtual_batch_size=52,
        #weights=1,
        #drop_last=False,
    #) 

    
    ## UNCOMMENT TO VALIDATE KAN MODELS AND E-KAN

    ### E-KAN
    models,model_p,dataset,dataset_p=KAN_tr_m(X_tr,y_tr.astype('int64'),X_v,y_v.astype('int64'),k=400,random_s=True,m='KAN',fil=True,subs=False,k1=4,k2=4,N=8)

    ### BASE AND DEEP KAN
    #dataset= {}
    #dataset['train_input'] = torch.from_numpy(X_tr.values)
    #dataset['test_input'] = torch.from_numpy(X_v.values)
    #dataset['train_label'] = torch.from_numpy(y_tr.values[:,0])
    #dataset['test_label'] = torch.from_numpy(y_v.values[:,0])

    #### DEEP

    #model_p = KAN(width=[dataset_p['train_input'].shape[1],8,4,2], grid=5, k=4)

    #def train_acc():
    #    return torch.mean((torch.argmax(model_p(dataset_p['train_input']), dim=1) == dataset_p['train_label']).float())

    #def test_acc():
    #    return torch.mean((torch.argmax(model_p(dataset_p['test_input']), dim=1) == dataset_p['test_label']).float())

    #results = model.train(dataset, opt="LBFGS", steps=50, metrics=(train_acc, test_acc), loss_fn=torch.nn.CrossEntropyLoss(),lamb=00.9),
    #results = model_p.train(dataset_p, opt="LBFGS", steps=20, metrics=(train_acc, test_acc), loss_fn=torch.nn.CrossEntropyLoss(),lamb=0.009)
    #results['train_acc'][-1], results['test_acc'][-1]

    #### BASE

    #model_p = KAN(width=[dataset_p['train_input'].shape[1],1,2], grid=5, k=4)

    #def train_acc():
    #    return torch.mean((torch.argmax(model_p(dataset_p['train_input']), dim=1) == dataset_p['train_label']).float())

    #def test_acc():
    #    return torch.mean((torch.argmax(model_p(dataset_p['test_input']), dim=1) == dataset_p['test_label']).float())

    #results = model.train(dataset, opt="LBFGS", steps=50, metrics=(train_acc, test_acc), loss_fn=torch.nn.CrossEntropyLoss(),lamb=00.9),
    #results = model_p.train(dataset_p, opt="LBFGS", steps=20, metrics=(train_acc, test_acc), loss_fn=torch.nn.CrossEntropyLoss())
    #results['train_acc'][-1], results['test_acc'][-1]


    # PREDICT

    # OTHERS
    #y_pred= model.predict(X_v.values) 

    # KAN
    y_pred=torch.argmax(model_p(dataset_p['test_input']), dim=1)



    # ESTIMATING OUTER FOLD METRICS
    acc = accuracy_score(y_v, y_pred)
    #bal_acc=balanced_accuracy_score(y_v, y_pred)
    f1=f1_score(y_v,y_pred)
    auc=roc_auc_score(y_v,y_pred)
    # store the result
    outer_results_acc.append(acc)
    #outer_results_bal_acc.append(bal_acc)
    outer_results_f1.append(f1)
    outer_results_auc.append(auc)
    # report progress
    print('>acc=%.3f' % (acc))
    #print('>bal_accacc=%.3f' % (bal_acc))
    print('>F1=%.3f' % (f1))
    print('>auc=%.3f' % (auc))

    fold_number+=1


print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results_acc), np.std(outer_results_acc)))
print('Accuracy results refer to mean and variance')

print('AUC: %.3f (%.3f)' % (np.mean(outer_results_auc), np.std(outer_results_auc)))
print('AUC results refer to mean and variance')

print('F1: %.3f (%.3f)' % (np.mean(outer_results_f1), np.std(outer_results_f1)))
print('F1 results refer to mean and variance')

#models=list(scores[0].get('estimator'))
''' 

#DEBUGGER TEST WETHER THE GS MODIFY THE CV WHEN FIT IS APPLIED 
for i in range(len(controllo)):
    for j in range(len(controllo[i][0])):
        if controllo[i][0][j] != prova[i][0][j]:
            break

if i == len(controllo)-1:
    if j == len(controllo[i][0])-1:
        print("The arrays are equal")
else:
    print("The arrays are not equal")
    print(controllo[i][0][j])
    print(prova[i][0][j])

 ''' 


# T-TEST

In [ ]:
from scipy.stats import ttest_ind_from_stats

def perform_t_test_with_stats(mean1, std1, n1, mean2, std2, n2, equal_var=True):
    """
    Perform an independent two-sample t-test using sample statistics.
    
    Parameters:
    - mean1: Mean of the first sample
    - std1: Standard deviation of the first sample
    - n1: Size of the first sample
    - mean2: Mean of the second sample
    - std2: Standard deviation of the second sample
    - n2: Size of the second sample
    - equal_var: Boolean indicating whether to assume equal variances (default: True)
    
    Returns:
    - t_stat: The t-test statistic
    - p_value: The p-value of the test
    """
    # Perform the t-test using the sample statistics
    t_stat, p_value = ttest_ind_from_stats(mean1, std1, n1, mean2, std2, n2, equal_var=equal_var)
    
    return t_stat, p_value

# Example usage
mean1 = 0.662  # Mean of the first sample
std1 = 0.061   # Standard deviation of the first sample
n1 = 10      # Size of the first sample

mean2 = 0.542  # Mean of the second sample
std2 = 0.085   # Standard deviation of the second sample
n2 = 10      # Size of the second sample

# Perform the t-test
t_stat, p_value = perform_t_test_with_stats(mean1, std1, n1, mean2, std2, n2)
print(f"T-test statistic: {t_stat}")
print(f"P-value: {p_value}")

if p_value < 0.05:
    print("Reject the null hypothesis: There is a significant difference between the two means.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference between the two means.")
